# Import library

In [1]:
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
import math


# Image segmentation

In [1]:

# Load the image containing the number plate
image = cv2.imread('sPlate.jpg')

# Convert the image to grayscale for easier character detection
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply thresholding to make characters stand out
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Find contours in the thresholded image
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Initialize a list to store bounding box coordinates
bounding_boxes = []
count = 0
# Iterate through the detected contours
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    
    # Filter out small contours as noise
    if w > 10 and h > 10:
        bounding_boxes.append((x, y, x + w, y + h))

# Draw bounding boxes around the characters
for (x1, y1, x2, y2) in bounding_boxes:
    count = count+1
    character = image[y1:y2, x1:x2]
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
#     cv2.imwrite('./temp/'+str(count)+'.jpg', character)

# Display the image with bounding boxes
cv2.imshow('Contour', thresh)
cv2.imshow('Image with Bounding Boxes', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


# Number Plate Detection

In [12]:

harcascade = "./model/cascade.xml"

cap = cv2.VideoCapture('./video/2.mp4')

cap.set(3, 640) # width
cap.set(4, 480) #height

min_area = 500
count = 0

while True:
    success, img = cap.read()
    if success:
        plate_cascade = cv2.CascadeClassifier(harcascade)
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        plates = plate_cascade.detectMultiScale(img_gray, 1.1, 4)

        for (x,y,w,h) in plates:
            area = w * h

            if area > min_area:
                cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 2)
                cv2.putText(img, "Number Plate", (x,y-5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 0, 255), 2)

                img_roi = img[y: y+h, x:x+w]
                cv2.imshow("ROI", img_roi)
#                 cv2.imwrite("plates/scaned_img_" + str(count) + ".jpg", img_roi)



        cv2.imshow("Result", img)
        cv2.resizeWindow('Result', 1300, 680)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break;
            
            # Skip every 3rd frame
        if count % 3 != 0:
            for _ in range(2):  # Skip the next 2 frames
                ret, _ = cap.read()
                if not ret:
                    break

        count += 1
    else:
        cv2.destroyAllWindows()
        break;


# Number Plate Character Detection

In [4]:
from tensorflow import keras
import cv2
import numpy as np

# Load the image containing the number plate
image = cv2.imread('sPlate.jpg')
numberPlateText = ''
# Convert the image to grayscale for easier character detection
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply thresholding to make characters stand out
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Find contours in the thresholded image
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# Initialize a list to store bounding box coordinates
bounding_boxes = []
count = 0
digits = list(map(str, range(10))) + ["ba", "pa"]
## load the trained model
model = keras.models.load_model('number_plate_model.h5')

### sort the contour by distance
dist = []
newContours = []
for point in contours:
    x, y, w, h = cv2.boundingRect(point)
    if w > 10 and h > 10:
        newContours.append(point)
        distance = int((x + (y+h)*1.5)/2)
        dist.append(distance)

sorted_points = sorted(dist)

# Iterate through the detected contours
for pp in sorted_points:
    x, y, w, h = cv2.boundingRect(newContours[dist.index(pp)])
    
    # Filter out small contours as noise
    if True:
        bounding_boxes.append((x, y, x + w, y + h))
        count = count+1
        input_image = image[y:y+h, x:x+w]
        cv2.imwrite('./temp/'+str(count)+'.jpg', input_image)
        
        input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB (if needed)
        input_image = cv2.resize(input_image, (128, 128))  # Resize the image to match the model's input size
        input_image = input_image / 255.0  # Normalize pixel values to the range [0, 1]

        # Add a batch dimension to the input image
        input_image = np.expand_dims(input_image, axis=0)

        # Make predictions
        predictions = model.predict(input_image)

        # Process the predictions as needed
        # For example, you can get the predicted class label or probabilities
        predicted_class = np.argmax(predictions)
        class_probabilities = predictions[0]
        numberPlateText = numberPlateText + digits[predicted_class]
        print("Predicted Class:", digits[predicted_class])
#         print("Class Probabilities:", class_probabilities)



# # Draw bounding boxes around the characters
# for (x1, y1, x2, y2) in bounding_boxes:
#     count = count+1
#     character = image[y1:y2, x1:x2]
#     cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
#     cv2.imwrite('./temp/'+str(count)+'.jpg', character)

# Display the image with bounding boxes
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.5
font_color = (0, 255, 0)  # BGR color (red in this case)
font_thickness = 2
cv2.putText(image, numberPlateText, (0,15), font, font_scale, font_color, font_thickness)
cv2.imshow('Image with Bounding Boxes', image)
print('bounding box', bounding_boxes)
cv2.imwrite('./temp/'+str(12)+'.jpg', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


1/1 [==============================] - 0s 266ms/step
Predicted Class: ba
1/1 [==============================] - 0s 62ms/step
Predicted Class: 1
1/1 [==============================] - 0s 89ms/step
Predicted Class: 8
1/1 [==============================] - 0s 83ms/step
Predicted Class: pa
1/1 [==============================] - 0s 94ms/step
Predicted Class: 5
1/1 [==============================] - 0s 88ms/step
Predicted Class: 8
1/1 [==============================] - 0s 76ms/step
Predicted Class: 6
1/1 [==============================] - 0s 94ms/step
Predicted Class: 4
bounding box [(17, 15, 39, 46), (40, 15, 51, 43), (51, 14, 63, 40), (64, 13, 82, 43), (11, 47, 33, 96), (30, 48, 49, 89), (54, 47, 73, 94), (76, 48, 95, 92)]
